In [1]:
import urllib
import requests
from datetime import datetime
import json
import pickle

URL = 'https://st-dev-data-api.azurewebsites.net'
invertersURL = urllib.parse.urljoin(URL, '/api/v0.1/buildings/energyville1/pv/roof/inverters/')
powerURL = urllib.parse.urljoin(URL, '/api/v0.1/buildings/energyville1/pv/roof/power/')
user = 'eric_massip'
pswd = 'ssCpoDC3uHF4ssHJtvMf'

DATE_FORMAT_STR = "%Y-%m-%d"

start_time = datetime(2017, 6, 1)
end_time = datetime(2017, 6, 2)


def get_inverter_ids():
    inverter_ids_json_response = json.loads(requests.get(invertersURL, auth=(user, pswd)).text)['data']
    inverter_ids = []
    for i in range(len(inverter_ids_json_response)):
        inverter_ids.append(inverter_ids_json_response[i]['InverterId'])
    return inverter_ids


def get_params(inverter_id, start, end):
    return {'start': start.strftime(DATE_FORMAT_STR),
            'end': end.strftime(DATE_FORMAT_STR),
            'time_zone': 'Central European Standard Time',
            'inverter_ids': inverter_id}


def get_power_consumption(inverter_id, start, end):
    return json.loads(requests.get(powerURL, params=get_params(inverter_id, start, end), auth=(user, pswd)).text)['data']


def get_power_consumptions(start, end):
    return [get_power_consumption(inverter_id, start, end) for inverter_id in get_inverter_ids()]


power_consumptions = get_power_consumptions(start_time, end_time)
#pickle.dump(power_consumptions, open('power_consumptions', 'wb'))


In [3]:
import pprint
from datetime import datetime

sum = 0
for inverter_consumption in power_consumptions:
    for piece in inverter_consumption:
        hour = datetime.fromisoformat(piece['DateTimeMeasurement']).hour
        minute = datetime.fromisoformat(piece['DateTimeMeasurement']).minute
        if piece['GenerationPower'] > 0 and minute == 0 and hour == 10:
            sum += piece['GenerationPower']
            pprint.pprint(piece)

print('SUM: ' + str(sum))

{'DCGenerationPower': 11439.0,
 'DateTimeMeasurement': '2017-06-01 10:00:00+02:00',
 'GenerationPower': 11251.0,
 'InverterDescription': 'Omvormer 01',
 'InverterId': '9369356'}
{'DCGenerationPower': 10512.0,
 'DateTimeMeasurement': '2017-06-01 10:00:00+02:00',
 'GenerationPower': 10238.0,
 'InverterDescription': 'Omvormer 02',
 'InverterId': '9363524'}
{'DCGenerationPower': 10820.0,
 'DateTimeMeasurement': '2017-06-01 10:00:00+02:00',
 'GenerationPower': 10592.0,
 'InverterDescription': 'Omvormer 04',
 'InverterId': '9363686'}
{'DCGenerationPower': 10743.0,
 'DateTimeMeasurement': '2017-06-01 10:00:00+02:00',
 'GenerationPower': 10541.0,
 'InverterDescription': 'Omvormer 05',
 'InverterId': '11637887'}
{'DCGenerationPower': 2750.0,
 'DateTimeMeasurement': '2017-06-01 10:00:00+02:00',
 'GenerationPower': 4500.0,
 'InverterDescription': 'Omvormer 06',
 'InverterId': '9363848'}
{'DCGenerationPower': 2720.0,
 'DateTimeMeasurement': '2017-06-01 10:00:00+02:00',
 'GenerationPower': 4500.0,
